# Sector Visualization

In this example, we will parse the database and visualize each sector's averaage times.

## Step 0: Global variables

In [1]:
database_path = 'bike_data.db'

## Step 1: Import required libraries and the database

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from ipyleaflet import Map, Marker, Polygon
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm

# SQLAlchemy connectable 
conn = create_engine( 'sqlite:///' + database_path ).connect()

## Step 2: Import Sector Data

In [3]:
sector_list = pd.read_sql_table( 'sector_list',conn)
centroid = [0, 0]
num_points = 0

polygons = []

counter = 0.0
for sector in sector_list['sector_id']:
    
    sector_points = pd.read_sql_table( sector, conn )
    
    poly_draw = Polygon( )
    centroid = [centroid[0] + sector_points['latitude'].mean()  * len(sector_points),
                centroid[1] + sector_points['longitude'].mean() * len(sector_points)]
    num_points += len( sector_points )
    
    color_code = matplotlib.colors.rgb2hex(plt.get_cmap('hsv')( counter / len(sector_list) ))
    poly_points = [ ( row['latitude'], row['longitude'] ) for index, row in sector_points.iterrows() ]
    polygons.append( Polygon( locations=poly_points, 
                              color=color_code,
                              fill_color=color_code ) )
    counter += 1.0

Here is a handy visualization of the designated sectors.

In [4]:
centroid_pt = (centroid[0]/num_points, 
               centroid[1]/num_points)

#  Build Map Visualization    
m = Map( center=centroid_pt, zoom=11)
for polygon in polygons:
    m.add_layer(polygon);
    
m.layout.height="550px"    
m 

Map(center=[39.68022141177373, -104.91524623127385], controls=(ZoomControl(options=['position', 'zoom_in_text'…

For each sector, we need to construct tracks

In [5]:
def Build_Segment( conn, point_list, dataset_id, sector_id ):
    
    #  Get a list of all points with that sector-id
    sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {})'.format( sector_id, dataset_id )
    points_in_sector = pd.read_sql_query( sql_query, conn )
    print(points_in_sector)
    
    return points_in_sector

Construct the tracks

In [6]:
#  For each segment, we need to create a track for each dataset
dataset_ids = pd.read_sql_query('SELECT DISTINCT dataset_id FROM point_list', conn)

#  Load the entire list of points into one dataframe for quicker access
point_list = pd.read_sql_table( 'point_list', conn )

for sector_id in sector_list['sector_id']:
    
    #  For each dataset
    for dataset_id in dataset_ids['dataset_id']:
        
        #  Create a full track of the segment
        sql_query = 'SELECT * FROM point_list WHERE (sector_id = \'{}\') and (dataset_id = {})'.format( sector_id, dataset_id )
        points_in_sector = pd.read_sql_query( sql_query, conn )
        
        #  from the point list, we can compute time spent in sector
        points_in_sector['timestamp'] =  pd.to_datetime(points_in_sector['timestamp'])
        time_in_sector = points_in_sector['timestamp'].max() - points_in_sector['timestamp'].min()
        print('Sector: {}, Dataset: {}, Time: {}'.format( sector_id, dataset_id, time_in_sector))
        

Sector: sector_0, Dataset: 0.0, Time: 0 days 00:09:54
Sector: sector_0, Dataset: 1.0, Time: NaT
Sector: sector_1, Dataset: 0.0, Time: 0 days 00:08:24
Sector: sector_1, Dataset: 1.0, Time: 0 days 00:07:08
Sector: sector_2, Dataset: 0.0, Time: 0 days 00:07:17
Sector: sector_2, Dataset: 1.0, Time: 0 days 00:07:43
Sector: sector_3, Dataset: 0.0, Time: 0 days 00:09:09
Sector: sector_3, Dataset: 1.0, Time: 0 days 00:08:35
Sector: sector_4, Dataset: 0.0, Time: 0 days 00:08:13
Sector: sector_4, Dataset: 1.0, Time: 0 days 00:07:32
Sector: sector_5, Dataset: 0.0, Time: 0 days 00:09:38
Sector: sector_5, Dataset: 1.0, Time: 0 days 00:09:01
Sector: sector_6, Dataset: 0.0, Time: 0 days 00:04:50
Sector: sector_6, Dataset: 1.0, Time: 0 days 00:04:46
Sector: sector_7, Dataset: 0.0, Time: 0 days 00:07:15
Sector: sector_7, Dataset: 1.0, Time: 0 days 00:07:43
Sector: sector_8, Dataset: 0.0, Time: 0 days 00:06:25
Sector: sector_8, Dataset: 1.0, Time: 0 days 00:06:30
